In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro, levene, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# 파일을 읽기
all_features = pd.read_csv('C:/Users/SAMSUNG/Documents/vscode/2024/ML/프로젝트2/data/윈저라이징_inf제거.csv',index_col=0)
all_features = all_features.drop(columns = ['코스닥연수익률 (%)', '코스피연수익률(%)','코스닥변동성', '코스피변동성'])
all_features

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_15404\648810436.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,회사명,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
0,(주)ES큐브,50120,2012,-16.575097,0.0,92.510776,1.564118,4.854248,-0.4200,28.015517,...,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008,0.0
1,(주)ES큐브,50120,2013,1.849357,0.0,-1427.915750,6.304012,4.692085,-53.5257,24.759975,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,0.0
2,(주)ES큐브,50120,2014,20.099174,0.0,96.260255,17.123515,-24.126991,-53.5257,21.419545,...,3.6,353600472,2.13,1099.3,-5.961831,-8.166371,2.098,101.12,-0.531182,0.0
3,(주)ES큐브,50120,2015,-1.751499,0.0,-274.898482,0.053133,-5.188115,44.4400,27.803407,...,3.5,358513845,1.67,1172.5,-5.217313,-18.112831,1.662,97.06,-4.015032,0.0
4,(주)ES큐브,50120,2016,89.076046,0.0,1033.661139,44.390095,-18.654872,-23.0800,28.682162,...,3.4,361701404,1.52,1207.7,-3.297016,-4.409925,1.638,95.30,-1.813311,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12317,한화화인케미칼(주),25850,2013,0.000000,0.0,0.000000,0.000000,0.000000,-6.8600,-18.891623,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,1.0
12318,핸즈코퍼레이션(주),143210,2022,-3.617503,0.0,-21.656986,-25.683177,38.426889,-24.2500,4.960248,...,3.0,399043064,3.98,1264.5,16.641876,20.573937,3.722,115.29,8.375635,1.0
12319,현대로템(주),64350,2015,14.613589,0.0,-1427.915750,-17.312414,3.698453,-1.4100,-0.362243,...,3.5,358513845,1.67,1172.5,-5.217313,-18.112831,1.662,97.06,-4.015032,1.0
12320,현대페인트(주),11720,2012,0.000000,0.0,0.000000,0.000000,0.000000,-1.4500,22.162032,...,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008,1.0


In [3]:
from sklearn.preprocessing import MinMaxScaler

# Drop non-feature columns and separate features and target
X = all_features.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
y = all_features['부실']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=777)

# 데이터 표준화 - Min-Max Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 표준화된 데이터를 DataFrame으로 변환
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# y_train 인덱스 정렬
y_train = y_train.reset_index(drop=True)

# '0'과 '1' 집단으로 나누기
group0_train = X_train_scaled[y_train == 0]
group1_train = X_train_scaled[y_train == 1]

# 결과 저장용 리스트
results = []

# 각 피처에 대해 테스트 수행
for feature in X_train_scaled.columns:
    dw_0 = durbin_watson(group0_train[feature])
    dw_1 = durbin_watson(group1_train[feature])
    shapiro_0 = shapiro(group0_train[feature])
    shapiro_1 = shapiro(group1_train[feature])
    levene_test = levene(group0_train[feature], group1_train[feature])
    t_test = ttest_ind(group0_train[feature], group1_train[feature])
    
    results.append({
        'feature': feature,
        'dw_0': dw_0,
        'dw_1': dw_1,
        'shapiro_0_stat': shapiro_0.statistic,
        'shapiro_0_p': shapiro_0.pvalue,
        'shapiro_1_stat': shapiro_1.statistic,
        'shapiro_1_p': shapiro_1.pvalue,
        'levene_stat': levene_test.statistic,
        'levene_p': levene_test.pvalue,
        't_stat': t_test.statistic,
        't_p': t_test.pvalue
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)
results_df

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_15404\3716664383.py:33: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 7913.
  shapiro_0 = shapiro(group0_train[feature])


,feature,dw_0,dw_1,shapiro_0_stat,shapiro_0_p,shapiro_1_stat,shapiro_1_p,levene_stat,levene_p,t_stat,t_p
0,총자본증가율,0.210603,0.479504,0.656119,5.471390e-83,0.746324,1.341791e-31,81.890366,1.753288e-19,3.913010,9.185341e-05
1,정상영업이익증가율,0.700831,0.361740,0.539717,1.267566e-89,0.254810,1.743446e-46,74.971340,5.640187e-18,4.449521,8.714199e-06
2,순이익증가율,0.072730,0.139429,0.594797,1.085501e-86,0.643808,5.770771e-36,21.729630,3.186187e-06,11.345770,1.261792e-29
3,자기자본증가율,0.237476,0.667466,0.570081,4.780532e-88,0.716632,5.447997e-33,166.258579,1.081810e-37,4.403477,1.078145e-05
4,매출액증가율,0.244076,0.487977,0.701731,7.620561e-80,0.792542,3.761905e-29,61.079386,6.126422e-15,4.228016,2.381947e-05
...,...,...,...,...,...,...,...,...,...,...,...
62,수출물가지수증감률,1.054872,1.156845,0.771860,4.319352e-74,0.728942,1.987650e-32,31.420064,2.142259e-08,4.157332,3.251020e-05
63,수입물가지수증감률,0.495586,0.425529,0.935395,5.781572e-50,0.930364,1.122421e-17,72.142397,2.335951e-17,2.166732,3.028262e-02
64,3년국고채수익률,0.661117,0.524325,0.885380,2.525156e-60,0.920055,5.855233e-19,2.797547,9.444494e-02,-4.933564,8.225260e-07
65,생산자물가지수,0.751225,0.592669,0.813332,6.278579e-70,0.821262,2.097865e-27,12.572898,3.934570e-04,-0.231862,8.166505e-01


In [4]:
# 정규성 검정에서 유의한 피처 구분
significant_normal_features2 = results_df[(results_df['shapiro_0_p'] > 0.05) & (results_df['shapiro_1_p'] > 0.05)]
significant_non_normal_features2 = results_df[(results_df['shapiro_0_p'] <= 0.05) | (results_df['shapiro_1_p'] <= 0.05)]

# 유의한 피처의 개수 출력
print("정규성을 충족하는 유의한 피처의 개수:", len(significant_normal_features2))
print("정규성을 충족하지 않는 유의한 피처의 개수:", len(significant_non_normal_features2))

# 유의한 피처의 이름과 p-value 값 출력
print("\n정규성을 충족하는 유의한 피처:")
print(significant_normal_features2[['feature', 'shapiro_0_p', 'shapiro_1_p']])

print("\n정규성을 충족하지 않는 유의한 피처:")
print(significant_non_normal_features2[['feature', 'shapiro_0_p', 'shapiro_1_p']])

정규성을 충족하는 유의한 피처의 개수: 0
정규성을 충족하지 않는 유의한 피처의 개수: 67

정규성을 충족하는 유의한 피처:
Empty DataFrame
Columns: [feature, shapiro_0_p, shapiro_1_p]
Index: []

정규성을 충족하지 않는 유의한 피처:
       feature   shapiro_0_p   shapiro_1_p
0       총자본증가율  5.471390e-83  1.341791e-31
1    정상영업이익증가율  1.267566e-89  1.743446e-46
2       순이익증가율  1.085501e-86  5.770771e-36
3      자기자본증가율  4.780532e-88  5.447997e-33
4       매출액증가율  7.620561e-80  3.761905e-29
..         ...           ...           ...
62   수출물가지수증감률  4.319352e-74  1.987650e-32
63   수입물가지수증감률  5.781572e-50  1.122421e-17
64    3년국고채수익률  2.525156e-60  5.855233e-19
65     생산자물가지수  6.278579e-70  2.097865e-27
66  생산자물가지수증감률  1.083099e-57  4.601013e-23

[67 rows x 3 columns]


In [5]:
# 등분산성 검정에서 유의한 피처 구분
significant_normal_features = results_df[(results_df['levene_p'] > 0.05)]
significant_non_normal_features = results_df[(results_df['levene_p'] <= 0.05)]

# 유의한 피처의 개수 출력
print("등분산성을 충족하는 유의한 피처의 개수:", len(significant_normal_features))
print("등분산성을 충족하지 않는 유의한 피처의 개수:", len(significant_non_normal_features))

# 유의한 피처의 이름과 p-value 값 출력
print("\n등분산성을 충족하는 유의한 피처:")
print(significant_normal_features[['feature', 'levene_p']])

print("\n등분산성을 충족하지 않는 유의한 피처:")
print(significant_non_normal_features[['feature', 'levene_p']])

등분산성을 충족하는 유의한 피처의 개수: 18
등분산성을 충족하지 않는 유의한 피처의 개수: 49

등분산성을 충족하는 유의한 피처:
          feature  levene_p
14         총자본회전률  0.160619
17        유동자산회전률  0.343296
18        당좌자산회전률  0.422852
19        재고자산회전률  0.238065
20        매출채권회전률  0.227317
22        운전자본회전률  0.464003
23    종업원1인당 부가가치  0.336039
25          기계장비율  0.195622
26          자본집약도  0.670936
28         설비투자효율  0.169054
32         기계투자효율  0.076372
36           유동비율  0.379733
37           당좌비율  0.118419
38           현금비율  0.157759
40  매출채권 대 매입채무비율  0.190283
53    GDP대비국가부채비율  0.512352
60          CD이자율  0.064559
64       3년국고채수익률  0.094445

등분산성을 충족하지 않는 유의한 피처:
              feature       levene_p
0              총자본증가율   1.753288e-19
1           정상영업이익증가율   5.640187e-18
2              순이익증가율   3.186187e-06
3             자기자본증가율   1.081810e-37
4              매출액증가율   6.126422e-15
5             종업원수증가율   2.034341e-35
6             매출액총이익률   1.193842e-02
7          매출액정상영업이익률  2.000550e-235
8             매출액순이익률   0.000000e+00
